In [1]:
import random
import pandas as pd
import Covid19_model as CM
import numpy as np
import shutil
import sys
import os.path
import math
import csv
import lhsmdu
import time
from Pyomo_solve import solve_ode

# Set parameter

In [2]:
D = 1e2
n_site = 16
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.5e6
length = 50

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne = 0, 0, 0
done = False
index = [0]
start_time = 6
aa = [1] * 5
for t in range(T):
    p_set = env.para_sampling(100)
    para = p_set[index[0]]
    action_buffer = [] 
    reward_buffer = []
    s = np.array(state)
    ac = []
    if t > start_time:
        for a in range(n_site):
            try:
                aa = solve_ode(s[:,a], np.sum(s[1:-1,a])*budget/np.sum(s[1:-1,:]), para, length)
            except Exception:
                print("pyomo fail")
            ac.append(aa)
        action = np.array(ac).T
        if budget < 0: 
            action = np.zeros((5,n_site))
    else:
        action = np.zeros((5,n_site))
        
    
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, p_set, 100, 1)
    #if not done and t > start_time: 
     #   env.space_update(ck, p_set, n_select)
    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e

    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'True State Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, score:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]),
                  budget, score))
    print('\r', 'Observe Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]),
                  np.sum(state[4]), np.sum(state[5]))) 
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 True State Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2500000.00, score:-229.27
 Observe Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0
 Time: 0 | new_e:52.9, cum_e:52.9, new_i:47.3, cum_i:47.3
 True State Time: 1 | S: 25495799.8, E: 147.8, A: 0.0, I: 47.3, D: 0.0, R: 5.1, Budget: 2500000.00, score:-554.35
 Observe Time: 1 | S: 25495797.0, E: 143.0, A: 0.0, I: 47.3, D: 0.0, R: 0.0
 Time: 1 | new_e:74.3, cum_e:127.2, new_i:65.4, cum_i:112.6
 True State Time: 2 | S: 25495676.0, E: 199.6, A: 0.0, I: 112.6, D: 0.1, R: 11.8, Budget: 2500000.00, score:-1016.39
 Observe Time: 2 | S: 25495678.0, E: 199.0, A: 0.0, I: 112.6, D: 0.1, R: 0.0
 Time: 2 | new_e:104.9, cum_e:232.1, new_i:91.3, cum_i:204.0
 True State Time: 3 | S: 25495501.2, E: 274.0, A: 0.0, I: 203.8, D: 0.2, R: 20.8, Budget: 2500000.00, score:-1674.21
 Observe Time: 3 | S: 25495510.0, E: 273.0, A: 0.0, I: 203.8, D: 0.2, R: 12.0
 Time: 3 | new_e:148.7, cum_e:380.8, new_i:128.5, cum_

pyomo fail
 True State Time: 18 | S: 24688795.2, E: 34.5, A: 505.9, I: 1229.3, D: 4.7, R: 805430.4, Budget: 1847025.23, score:-11425.33
 Observe Time: 18 | S: 24688907.0, E: 25.0, A: 505.9, I: 1229.3, D: 4.7, R: 805427.0
 Time: 18 | new_e:15.7, cum_e:2569.2, new_i:68.4, cum_i:3262.8
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
pyomo fail
 True State Time: 19 | S: 24613676.6, E: 18.1, A: 455.4, I: 1143.9, D: 4.9, R: 880701.0, Budget: 1791565.64, score:-11469.44
 Observe Time: 19 | S: 24613794.0, E: 18.0, A: 455.4, I: 1143.9, D: 4.9, R: 880697.0
 Time: 19 | new_e:8.2, cum_e:2577.4, new_i:56.2, cum_i:3319.0
pyomo fail
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 20 | S: 24554049.6, E: 8

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 39 | S: 23581549.4, E: 10.7, A: 45.8, I: 213.5, D: 6.6, R: 1914174.1, Budget: 1126619.68, score:-12275.65
 Observe Time: 39 | S: 23581778.0, E: 0.0, A: 45.8, I: 213.5, D: 6.6, R: 1914171.0
 Time: 39 | new_e:4.6, cum_e:2741.7, new_i:9.1, cum_i:3794.7
 True State Time: 40 | S: 23534626.2, E: 10.9, A: 41.6, I: 196.0, D: 6.6, R: 1961118.8, Budget: 1101987.77, score:-12297.48
 Observe Time: 40 | S: 23534864.0, E: 0.0, A: 41.6, I: 196.0, D: 6.6, R: 1961116.0
 Time: 40 | new_e:4.7, cum_e:2746.4, new_i:8.8, cum_i:3803.4
 True State Time: 41 | S: 23505866.8, E: 10.7, A: 38.5, I: 180.2, D: 6.6, R: 1989897.2, Budget: 1076321.38, score:-12318.80
 Observe Time: 41 | S: 23506110.0, E: 0.0, A: 38.5, I: 180.2, D: 6.6, R: 1989896.0
 Time: 41 | new_e:4.6, cum_e:2751.0, new_i:8.3, cum_i:3811.8
    mo

pyomo fail
 True State Time: 60 | S: 22852282.8, E: 19.7, A: 25.4, I: 83.7, D: 6.9, R: 2643581.5, Budget: 640445.57, score:-12907.76
 Observe Time: 60 | S: 22852706.0, E: 0.0, A: 25.4, I: 83.7, D: 6.9, R: 2643579.0
 Time: 60 | new_e:8.3, cum_e:2882.5, new_i:10.3, cum_i:3991.6
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 61 | S: 22822890.7, E: 22.1, A: 24.4, I: 83.5, D: 6.9, R: 2672972.3, Budget: 621882.32, score:-12948.40
 Observe Time: 61 | S: 22823327.0, E: 0.0, A: 24.4, I: 83.5, D: 6.9, R: 2672970.0
 Time: 61 | new_e:9.2, cum_e:2891.8, new_i:11.1, cum_i:4002.8
 True State Time: 62 | S: 22795289.7, E: 23.9, A: 24.7, I: 84.2, D: 7.0, R: 2700570.5, Budget: 603822.63, score:-12992.22
 Observe Time: 62 | S: 22795740.0, E: 0.0, A: 24.7, I: 84.2, D: 7.0, R: 2700572.0
 Time: 62 | new_e:10.0, cum_e:2901.8, new_i:11.9, cum_i:4014.6


 True State Time: 78 | S: 22408335.0, E: 177.9, A: 97.9, I: 326.8, D: 7.4, R: 3087055.0, Budget: 345623.96, score:-15433.00
 Observe Time: 78 | S: 22409549.0, E: 0.0, A: 97.9, I: 326.8, D: 7.4, R: 3087027.0
 Time: 78 | new_e:72.5, cum_e:3459.1, new_i:77.9, cum_i:4598.4
 True State Time: 79 | S: 22390847.1, E: 211.8, A: 94.4, I: 363.8, D: 7.4, R: 3104475.5, Budget: 335708.85, score:-15809.36
 Observe Time: 79 | S: 22392181.0, E: 0.0, A: 94.4, I: 363.8, D: 7.4, R: 3104442.0
 Time: 79 | new_e:86.2, cum_e:3545.3, new_i:90.3, cum_i:4688.7
 True State Time: 80 | S: 22381031.1, E: 251.4, A: 94.4, I: 408.6, D: 7.4, R: 3114207.2, Budget: 324115.62, score:-16255.47
 Observe Time: 80 | S: 22382506.0, E: 0.0, A: 94.4, I: 408.6, D: 7.4, R: 3114167.0
 Time: 80 | new_e:102.3, cum_e:3647.6, new_i:105.2, cum_i:4793.9
 True State Time: 81 | S: 22364601.1, E: 300.7, A: 94.6, I: 462.7, D: 7.5, R: 3130533.5, Budget: 315824.58, score:-16788.57
 Observe Time: 81 | S: 22366243.0, E: 0.0, A: 94.6, I: 462.7, D:

In [6]:
#result_state = pd.DataFrame(state_buffer)
#result_state.to_csv(r'MPC_state_16.csv')
#result_new = pd.DataFrame(new_buffer)
#result_new.to_csv(r'MPC_new_16.csv')

In [7]:
(time.time() - tic) / ( T - start_time + 1)

98.36308525029351